In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pylab as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
CLASSIFIER_URL ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"
IMAGE_RES = 224

model = 

In [ ]:
import numpy as np
import PIL.Image as Image

filepath = 'https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg'
grace_hopper = 
grace_hopper = 
grace_hopper 

In [ ]:
# normalize pixel value to (0,1)
grace_hopper = 
grace_hopper.shape

In [ ]:
result = 
result.shape

In [ ]:
predicted_class = 
predicted_class

In [ ]:
l_name, l_path = 'ImageNetLabels.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'

labels_path = 
imagenet_labels = 

plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = 
_ = plt.title("Prediction: " + predicted_class_name.title())

In [ ]:
(train_examples, validation_examples), info = 

num_examples = 
num_classes = 

In [ ]:
for i, example_image in enumerate(train_examples.take(3)):
  print("Image {} shape: {}".format(i+1, example_image[0].shape))

In [ ]:
def format_image(image, label):
  
  return image, label

BATCH_SIZE = 32

train_batches      = 
validation_batches = 

In [ ]:
image_batch, label_batch = 
image_batch = 
label_batch = 

result_batch = 

predicted_class_names = 
predicted_class_names

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
  

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = 

In [ ]:
feature_batch = 
print(feature_batch.shape)

In [ ]:
# freeze layers of feature extractor
feature_extractor.trainable = 

In [ ]:
model = 

model.summary()

In [ ]:
# compile and train model
model.compile( )

EPOCHS = 6
history = model.fit( )

In [ ]:
acc = 
val_acc = 

loss = 
val_loss = 

epochs_range = 

plt.figure(figsize=(8, 8))



In [ ]:
class_names = 
class_names

In [ ]:
predicted_batch = 
predicted_batch = 
predicted_ids = 
predicted_class_names = 
predicted_class_names

In [ ]:
print("Labels: ", label_batch)
print("Predicted labels: ", predicted_ids)

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
